In [30]:
import logging
import json
import os
import re
from pathlib import Path
from collections import Counter

import anndata
import pandas as pd
import numpy as np
import tifffile
from einops import rearrange
from skimage.measure import regionprops
from skimage.morphology import label

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import multiplex_imaging_pipeline.utils as utils
import multiplex_imaging_pipeline.ome as ome
import multiplex_imaging_pipeline.segmentation as seg
import multiplex_imaging_pipeline.multiplex_imaging_pipeline as mip
from multiplex_imaging_pipeline.spatial_features import DEFAULT_GATING_STRATEGY

2024-03-26 08:06:27.245328: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /diskmnt/Projects/Users/estorrs/miniconda3/envs/imaging_analysis_v2/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/lib:/usr/lib64:/usr/local/lib:/usr/lib64:
2024-03-26 08:06:27.245386: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
parent_dir = f'/diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone'
seg_dir = os.path.join(parent_dir, 'segmentations')
Path(seg_dir).mkdir(parents=True, exist_ok=True)

In [21]:
seg_dir

'/diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations'

In [6]:
nuclei_channels = ['DAPI']
membrane_channels = [
    'Pan-Cytokeratin', 'HLA-DR', 'SMA', 'CD4', 'CD45', 'Hep-Par-1', 'CD31', 'E-cadherin', 'CD68', 'CD3e']

In [ ]:
2023-12-19_C3L-01287-11Us1	/diskmnt/primary/CODEX/HTAN/20231219_Human_CCRCC_C3L-01287-11Us1_2_top_C3N-01200-11Us1_2_middle_C3N-01200-13-U2_btm/C3L-01287-11Us1__20231219.ome.tiff
2023-07-01_HT565B1-S1H2_left	/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT206B1-S1H5_TOP_HT297B1-S1H1_HT339B2-H2_HT565B1-S1H2_BTTM/HT565B1-S1H2_left__20230701.ome.tiff
2023-07-01_HT565B1-S1H2_right	/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT206B1-S1H5_TOP_HT297B1-S1H1_HT339B2-H2_HT565B1-S1H2_BTTM/HT565B1-S1H2_right__20230701.ome.tiff
2023-04-20_HT308B1-S1H5A4	/diskmnt/primary/CODEX/HTAN/041923_BRCA_HT171B1-S1H9A1-4_top_HT243B1-S1H4A4_HT271B1-S1H6A5_HT308B1-S1H5A4_btm/HT308B1-S1H5A4__20230420.ome.tiff
2023-04-27_HT565B1-H2	/diskmnt/primary/CODEX/HTAN/042623_MULTIPLE_HTAN_HT413C1-K2_TOP_HT339B2-H1_HT553P1-H2_HT565B1-H2_BTM/HT565B1-H2__20230427.ome.tiff
2022-05-05_HT397B1-H2A2	/diskmnt/primary/CODEX/HTAN/20220505-HT397B1-H2A2/HT397B1-H2A2__20220505.ome.tiff
2022-04-29_HT397B1-H3A1	/diskmnt/primary/CODEX/HTAN/20220429-HT397B1-H3A1/HT397B1-H3A1__20220429.ome.tiff
2021-10-08_HT323B1-S1H3A4	/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S1H3A4/HT323B1-S1H3A4__20211008.ome.tiff
2023-08-26_mCRC_S15-32775_Met	/diskmnt/primary/CODEX/HTAN/20230826_mCRC_S16-32775_Primary_and_Met/mCRC_S16-32775_Met__20230826.ome.tiff
2023-08-24_mCRC_S16-38794_Primary	/diskmnt/primary/CODEX/HTAN/20230824_mCRC_S16-38794_Primary/mCRC_S16-38794_Primary__20230824.ome.tiff
2023-08-23_mCRC_S16-38794_Met	/diskmnt/primary/CODEX/HTAN/08232023_mCRC_S16-38794_Met/mCRC_S16-38794_Met__20230823.ome.tiff
2023-03-08_S15_1909_C2_Section3	/diskmnt/primary/CODEX/HTAN/20230309_CRC_Met_S15_1909_C2_Section3/S15_1909_C2_Section3__20230308.ome.tiff
2023-04-13_HT488C1-Th1K1Fp1	/diskmnt/primary/CODEX/HTAN/041323_CRC_HT488C1-Th1K1Fp1-U2/HT488C1-Th1K1Fp1__20230413.ome.tiff
2023-04-19_HT488C1-Th1K1Fp1-U14	/diskmnt/primary/CODEX/HTAN/041823_CRC_HT488C1-Th1K1Fp1-U14/HT488C1-Th1K1Fp1-U14__20230419.ome.tiff

In [7]:
fp_str = """/diskmnt/primary/CODEX/HTAN/20231220_Human_CCRCC_C3L-00970-13u2_top_C3N-00733-12U2-PA00002352U1_Btom/C3L-00970-13u2__20231220.ome.tiff
/diskmnt/primary/CODEX/HTAN/20231219_Human_CCRCC_C3L-01287-11Us1_2_top_C3N-01200-11Us1_2_middle_C3N-01200-13-U2_btm/C3L-01287-11Us1__20231219.ome.tiff
/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT206B1-S1H5_TOP_HT297B1-S1H1_HT339B2-H2_HT565B1-S1H2_BTTM/HT565B1-S1H2_left__20230701.ome.tiff
/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT206B1-S1H5_TOP_HT297B1-S1H1_HT339B2-H2_HT565B1-S1H2_BTTM/HT565B1-S1H2_right__20230701.ome.tiff
/diskmnt/primary/CODEX/HTAN/041923_BRCA_HT171B1-S1H9A1-4_top_HT243B1-S1H4A4_HT271B1-S1H6A5_HT308B1-S1H5A4_btm/HT308B1-S1H5A4__20230420.ome.tiff
/diskmnt/primary/CODEX/HTAN/042623_MULTIPLE_HTAN_HT413C1-K2_TOP_HT339B2-H1_HT553P1-H2_HT565B1-H2_BTM/HT413C1-K2__20230427.ome.tiff
/diskmnt/primary/CODEX/HTAN/042623_MULTIPLE_HTAN_HT413C1-K2_TOP_HT339B2-H1_HT553P1-H2_HT565B1-H2_BTM/HT339B2-H1__20230427.ome.tiff
/diskmnt/primary/CODEX/HTAN/042623_MULTIPLE_HTAN_HT413C1-K2_TOP_HT339B2-H1_HT553P1-H2_HT565B1-H2_BTM/HT553P1-H2__20230427.ome.tiff
/diskmnt/primary/CODEX/HTAN/042623_MULTIPLE_HTAN_HT413C1-K2_TOP_HT339B2-H1_HT553P1-H2_HT565B1-H2_BTM/HT565B1-H2__20230427.ome.tiff
/diskmnt/primary/CODEX/HTAN/041223_BRCA_HT397B1-S1H3A1-U22/HT397B1-S1H3A1-U22__20230413.ome.tiff
/diskmnt/primary/CODEX/HTAN/031623_BRCA_HT397B1-U2/HT397B1-S1H3A1-U2__20230315.ome.tiff
/diskmnt/primary/CODEX/HTAN/040623_BRCA_HT397B1-U31/HT397B1-S1H3A1-U31__20230407.ome.tiff
/diskmnt/primary/CODEX/HTAN/03172023_BRCA_HT397B1-U12/HT397B1-S1H3A1-U12__20230316.ome.tiff
/diskmnt/primary/CODEX/HTAN/20220505-HT397B1-H2A2/HT397B1-H2A2__20220505.ome.tiff
/diskmnt/primary/CODEX/HTAN/20220429-HT397B1-H3A1/HT397B1-H3A1__20220429.ome.tiff
/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S1H3A4/HT323B1-S1H3A4__20211008.ome.tiff
/diskmnt/primary/CODEX/HTAN/20230826_mCRC_S16-32775_Primary_and_Met/mCRC_S16-32775_Met__20230826.ome.tiff
/diskmnt/primary/CODEX/HTAN/20230824_mCRC_S16-38794_Primary/mCRC_S16-38794_Primary__20230824.ome.tiff
/diskmnt/primary/CODEX/HTAN/08232023_mCRC_S16-38794_Met/mCRC_S16-38794_Met__20230823.ome.tiff
/diskmnt/primary/CODEX/HTAN/20230309_CRC_Met_S15_1909_C2_Section3/S15_1909_C2_Section3__20230308.ome.tiff
/diskmnt/primary/CODEX/HTAN/041323_CRC_HT488C1-Th1K1Fp1-U2/HT488C1-Th1K1Fp1__20230413.ome.tiff
/diskmnt/primary/CODEX/HTAN/041823_CRC_HT488C1-Th1K1Fp1-U14/HT488C1-Th1K1Fp1-U14__20230419.ome.tiff"""
fps = fp_str.split('\n')
fps

['/diskmnt/primary/CODEX/HTAN/20231220_Human_CCRCC_C3L-00970-13u2_top_C3N-00733-12U2-PA00002352U1_Btom/C3L-00970-13u2__20231220.ome.tiff',
 '/diskmnt/primary/CODEX/HTAN/20231219_Human_CCRCC_C3L-01287-11Us1_2_top_C3N-01200-11Us1_2_middle_C3N-01200-13-U2_btm/C3L-01287-11Us1__20231219.ome.tiff',
 '/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT206B1-S1H5_TOP_HT297B1-S1H1_HT339B2-H2_HT565B1-S1H2_BTTM/HT565B1-S1H2_left__20230701.ome.tiff',
 '/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT206B1-S1H5_TOP_HT297B1-S1H1_HT339B2-H2_HT565B1-S1H2_BTTM/HT565B1-S1H2_right__20230701.ome.tiff',
 '/diskmnt/primary/CODEX/HTAN/041923_BRCA_HT171B1-S1H9A1-4_top_HT243B1-S1H4A4_HT271B1-S1H6A5_HT308B1-S1H5A4_btm/HT308B1-S1H5A4__20230420.ome.tiff',
 '/diskmnt/primary/CODEX/HTAN/042623_MULTIPLE_HTAN_HT413C1-K2_TOP_HT339B2-H1_HT553P1-H2_HT565B1-H2_BTM/HT413C1-K2__20230427.ome.tiff',
 '/diskmnt/primary/CODEX/HTAN/042623_MULTIPLE_HTAN_HT413C1-K2_TOP_HT339B2-H1_HT553P1-H2_HT565B1-H2_BTM/HT339B2-H1__20230427.ome.tiff',
 '/

In [8]:
for fp in fps:
    specimen = fp.split('/')[-1].replace('.ome.tiff', '')
    print(specimen)
    prefix = os.path.join(seg_dir, specimen)

    try:
        mip.segment_ome(fp, prefix, 25000, nuclei_channels, membrane_channels)
    except:
        print(f'{specimen} failed')

2024-03-26 08:06:54,896 - starting segmentation for /diskmnt/primary/CODEX/HTAN/08232023_mCRC_S16-38794_Met/mCRC_S16-38794_Met__20230823.ome.tiff
2024-03-26 08:06:54,897 - Checking for cached data


mCRC_S16-38794_Met__20230823


2024-03-26 08:06:56,015 - Checking MultiplexSegmentation-9.tar.gz against provided file_hash...
2024-03-26 08:06:56,017 - MultiplexSegmentation-9.tar.gz with hash a1dfbce2594f927b9112f23a0a1739e0 already available.
2024-03-26 08:06:56,017 - Extracting /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz
2024-03-26 08:06:58,913 - Successfully extracted /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz into /home/estorrs/.deepcell/models
2024-03-26 08:06:59.097194: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /diskmnt/Projects/Users/estorrs/miniconda3/envs/imaging_analysis_v2/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/lib:/usr/lib64:/usr/local/lib:/usr/lib64:
2024-03-26 08:06:59.097239: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-03-26

2024-03-26 08:07:06,344 - No training configuration found in save file, so the model was *not* compiled. Compile it manually.


2024-03-26 08:26:21,982 - spliting into nrows: 2, ncols: 2
2024-03-26 08:26:21,987 - 0 0
2024-03-26 08:26:21,988 - window: 0, 25000, 0, 25000
/diskmnt/Projects/Users/estorrs/miniconda3/envs/imaging_analysis_v2/lib/python3.10/site-packages/deepcell_toolbox/deep_watershed.py:108: UserWarning: h_maxima peak finding algorithm was selected, but the provided image is larger than 5k x 5k pixels.This will lead to slow prediction performance.
  warnings.warn('h_maxima peak finding algorithm was selected, '
2024-03-26 10:13:33,107 - 0 1
2024-03-26 10:13:33,113 - window: 0, 25000, 24800, 49800
2024-03-26 11:02:22,964 - 1 0
2024-03-26 11:02:22,969 - window: 24800, 49800, 0, 25000
2024-03-26 12:18:07,697 - 1 1
2024-03-26 12:18:07,707 - window: 24800, 49800, 24800, 49800
2024-03-26 12:50:13,280 - finished segmentation
2024-03-26 12:50:13,287 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/mCRC_S16-38794_Met__20230823_nuclei_segmentation.tif
2024-03-26 12:50:

...

/diskmnt/Projects/Users/estorrs/miniconda3/envs/imaging_analysis_v2/lib/python3.10/site-packages/deepcell_toolbox/deep_watershed.py:108: UserWarning: h_maxima peak finding algorithm was selected, but the provided image is larger than 5k x 5k pixels.This will lead to slow prediction performance.
  warnings.warn('h_maxima peak finding algorithm was selected, '
2024-03-26 16:22:47,043 - finished segmentation
2024-03-26 16:22:47,048 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/HT488C1-Th1K1Fp1__20230413_nuclei_segmentation.tif
2024-03-26 16:22:47,719 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/HT488C1-Th1K1Fp1__20230413_cell_segmentation.tif
2024-03-26 16:22:48,409 - starting segmentation for /diskmnt/primary/CODEX/HTAN/041823_CRC_HT488C1-Th1K1Fp1-U14/HT488C1-Th1K1Fp1-U14__20230419.ome.tiff
2024-03-26 16:22:48,411 - Checking for cached data


HT488C1-Th1K1Fp1-U14__20230419


2024-03-26 16:22:48,646 - Checking MultiplexSegmentation-9.tar.gz against provided file_hash...
2024-03-26 16:22:48,647 - MultiplexSegmentation-9.tar.gz with hash a1dfbce2594f927b9112f23a0a1739e0 already available.
2024-03-26 16:22:48,648 - Extracting /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz
2024-03-26 16:22:51,342 - Successfully extracted /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz into /home/estorrs/.deepcell/models


2024-03-26 16:23:00,209 - No training configuration found in save file, so the model was *not* compiled. Compile it manually.


/diskmnt/Projects/Users/estorrs/miniconda3/envs/imaging_analysis_v2/lib/python3.10/site-packages/deepcell_toolbox/deep_watershed.py:108: UserWarning: h_maxima peak finding algorithm was selected, but the provided image is larger than 5k x 5k pixels.This will lead to slow prediction performance.
  warnings.warn('h_maxima peak finding algorithm was selected, '
2024-03-26 16:36:42,817 - finished segmentation
2024-03-26 16:36:42,820 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/HT488C1-Th1K1Fp1-U14__20230419_nuclei_segmentation.tif
2024-03-26 16:36:45,140 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/HT488C1-Th1K1Fp1-U14__20230419_cell_segmentation.tif


In [24]:
# nuclei_channels = ['DAPI']
# membrane_channels = [
#     'Pan-Cytokeratin', 'HLA-DR', 'SMA', 'CD4', 'CD45', 'Hep-Par-1', 'CD31', 'E-cadherin', 'CD68', 'CD3e']

In [25]:
# R_CHANNEL_MAPPING = seg.R_CHANNEL_MAPPING

In [27]:
# channel_to_img = seg.extract_ome_tiff(fp)
    
# channel_to_img = {R_CHANNEL_MAPPING.get(k, k):v for k, v in channel_to_img.items()}
# nuclei_channels = [R_CHANNEL_MAPPING.get(k, k) for k in nuclei_channels]
# membrane_channels = [R_CHANNEL_MAPPING.get(k, k) for k in membrane_channels]

In [23]:
# # fp = '/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT206B1-S1H5_TOP_HT297B1-S1H1_HT339B2-H2_HT565B1-S1H2_BTTM/HT565B1-S1H2_left__20230701.ome.tiff'
# # fp = '/diskmnt/primary/CODEX/HTAN/07012023_BRCA_HT206B1-S1H5_TOP_HT297B1-S1H1_HT339B2-H2_HT565B1-S1H2_BTTM/HT565B1-S1H2_right__20230701.ome.tiff'
# fp = '/diskmnt/primary/CODEX/HTAN/20230824_mCRC_S16-38794_Primary/mCRC_S16-38794_Primary__20230824.ome.tiff'

# specimen = fp.split('/')[-1].replace('.ome.tiff', '')
# prefix = os.path.join(seg_dir, specimen)


# mip.segment_ome(fp, prefix, 25000, nuclei_channels, membrane_channels)




In [36]:
fps = sorted(utils.listfiles(seg_dir, regex=r'nuclei'))
fps

['/diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/C3L-00970-13u2__20231220_nuclei_segmentation.tif',
 '/diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/C3L-01287-11Us1__20231219_nuclei_segmentation.tif',
 '/diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/HT308B1-S1H5A4__20230420_nuclei_segmentation.tif',
 '/diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/HT323B1-S1H3A4__20211008_nuclei_segmentation.tif',
 '/diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/HT339B2-H1__20230427_nuclei_segmentation.tif',
 '/diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/HT397B1-H2A2__20220505_nuclei_segmentation.tif',
 '/diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/HT397B1-H3A1__20220429_nuclei_segmentation.tif',
 '/diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmenta

In [37]:
for fp in fps:
    print(fp)

/diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/C3L-00970-13u2__20231220_nuclei_segmentation.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/C3L-01287-11Us1__20231219_nuclei_segmentation.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/HT308B1-S1H5A4__20230420_nuclei_segmentation.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/HT323B1-S1H3A4__20211008_nuclei_segmentation.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/HT339B2-H1__20230427_nuclei_segmentation.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/HT397B1-H2A2__20220505_nuclei_segmentation.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/HT397B1-H3A1__20220429_nuclei_segmentation.tif
/diskmnt/Projects/Users/estorrs/imaging-analysis/results/subclone/segmentations/HT397B1-S1H3A1-U12__2023

In [33]:
count = 0
for fp in fps:
    img = tifffile.imread(fp)
    labeled = label(img)
    print(labeled.max(), fp.split('/')[-1])
    count += labeled.max()

193926 C3L-00970-13u2__20231220_nuclei_segmentation.tif
270082 C3L-01287-11Us1__20231219_nuclei_segmentation.tif
118967 HT308B1-S1H5A4__20230420_nuclei_segmentation.tif
287757 HT323B1-S1H3A4__20211008_nuclei_segmentation.tif
41146 HT339B2-H1__20230427_nuclei_segmentation.tif
171277 HT397B1-H2A2__20220505_nuclei_segmentation.tif
335316 HT397B1-H3A1__20220429_nuclei_segmentation.tif
175377 HT397B1-S1H3A1-U12__20230316_nuclei_segmentation.tif
149727 HT397B1-S1H3A1-U22__20230413_nuclei_segmentation.tif
196804 HT397B1-S1H3A1-U2__20230315_nuclei_segmentation.tif
113147 HT397B1-S1H3A1-U31__20230407_nuclei_segmentation.tif
127132 HT413C1-K2__20230427_nuclei_segmentation.tif
44261 HT488C1-Th1K1Fp1-U14__20230419_nuclei_segmentation.tif
67033 HT488C1-Th1K1Fp1__20230413_nuclei_segmentation.tif
58551 HT553P1-H2__20230427_nuclei_segmentation.tif
69763 HT565B1-H2__20230427_nuclei_segmentation.tif
106086 HT565B1-S1H2_left__20230701_nuclei_segmentation.tif
105317 HT565B1-S1H2_right__20230701_nuclei_seg

In [34]:
count

6775620